# Character prediction model with 5 classes (Cartman, Butters, Kyle, Stan, Others) 
Models tried Multinomial Naive Bayes, Random Forest, Logistic Regression, Support Vector Machine using CountVectorizer and TFIDFVectorizer.

In [121]:
import re, string
import numpy as np
import pandas as pd

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score
from sklearn.metrics import accuracy_score, classification_report

In [2]:
#semi pre-cleaned dataset
df = pd.read_csv('cleaned_all-seasons.csv')

In [123]:
df.shape

(70896, 5)

In [27]:
list(df)

['Season', 'Episode', 'Character', 'Line']

In [28]:
df["Character"].unique()

array(['Stan', 'Kyle', 'Chef', ..., 'Father Barnes', 'Cardinal Mallory',
       'Blind Man'], dtype=object)

In [3]:
#Create a new column to tag the data into 5 classes - Cartman(2), Butters(1), Kyle(3), Stan(4) and None(0)
#creating new colummn target_char

df['target_char'] = 'Others'
df.loc[df.Character == 'Butters', 'target_char'] = 'Butters'
df.loc[df.Character == 'Cartman', 'target_char'] = 'Cartman'
df.loc[df.Character == 'Kyle', 'target_char'] = 'Kyle'
df.loc[df.Character == 'Stan', 'target_char'] = 'Stan'

In [30]:
list(df)

['Season', 'Episode', 'Character', 'Line', 'target_char']

In [31]:
len(df)

70896

In [4]:
corpus = df.Line.tolist()

for line in range(len(corpus)):
    corpus[line] = re.sub('\\n', '', corpus[line].rstrip()).lower()
    corpus[line] = " ".join(word.strip(string.punctuation) for word in corpus[line].split())

###### Create the vectorizers

In [114]:
vectorizer1 = CountVectorizer(stop_words = 'english', ngram_range = (1,1), min_df = 5)

X1 = vectorizer1.fit_transform(corpus)
Y1 = df.target_char
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1, Y1, test_size=0.3, stratify=Y1, random_state=11)

In [115]:
vectorizer2 = TfidfVectorizer(stop_words = 'english', ngram_range = (1,1), min_df = 5)

X2 = vectorizer2.fit_transform(corpus)
Y2 = df.target_char
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.3, stratify=Y2, random_state=11)

### Multinomial models

In [96]:
mnb = MultinomialNB()

In [97]:
###Multinomial Naive Bayes fit on train data
mnb.fit(X1_train, Y1_train)
train_preds = mnb.predict(X1_train)
#training
print(classification_report(Y1_train, train_preds, np.unique(Y1_train)))

              precision    recall  f1-score   support

     Butters       0.70      0.10      0.17      1821
     Cartman       0.72      0.36      0.48      6842
        Kyle       0.72      0.15      0.25      4969
      Others       0.69      0.97      0.81     30619
        Stan       0.65      0.18      0.28      5376

    accuracy                           0.69     49627
   macro avg       0.69      0.35      0.40     49627
weighted avg       0.69      0.69      0.62     49627



In [101]:
###Multinomial Naive Bayes model on test data
#tesing 
test_preds = mnb.predict(X1_test)
print(classification_report(Y1_test, test_preds,np.unique(Y1_test))) 
print("Recall:",recall_score(Y1_test, test_preds, average = 'macro'))

              precision    recall  f1-score   support

     Butters       0.29      0.01      0.01       781
     Cartman       0.54      0.09      0.16      2932
        Kyle       0.42      0.03      0.06      2130
      Others       0.63      0.98      0.77     13122
        Stan       0.39      0.05      0.09      2304

    accuracy                           0.63     21269
   macro avg       0.46      0.23      0.22     21269
weighted avg       0.56      0.63      0.51     21269

Recall: 0.23176878131610135


In [99]:
###Multinomial Naive Bayes fit on train data
mnb.fit(X2_train, Y2_train)
train_preds2 = mnb.predict(X2_train)
#training
print(classification_report(Y2_train, train_preds2, np.unique(Y2_train)))

              precision    recall  f1-score   support

     Butters       0.00      0.00      0.00      1821
     Cartman       0.88      0.06      0.11      6842
        Kyle       0.74      0.02      0.05      4969
      Others       0.63      1.00      0.77     30619
        Stan       0.69      0.02      0.04      5376

    accuracy                           0.63     49627
   macro avg       0.59      0.22      0.19     49627
weighted avg       0.66      0.63      0.50     49627



In [102]:
###Multinomial Naive Bayes model on test data
#tesing 
test_preds2 = mnb.predict(X2_test)
print(classification_report(Y2_test, test_preds2,np.unique(Y2_test))) 
print("Recall:",recall_score(Y2_test, test_preds2, average = 'macro'))

              precision    recall  f1-score   support

     Butters       0.00      0.00      0.00       781
     Cartman       0.64      0.03      0.05      2932
        Kyle       0.72      0.02      0.03      2130
      Others       0.62      1.00      0.77     13122
        Stan       0.42      0.01      0.02      2304

    accuracy                           0.62     21269
   macro avg       0.48      0.21      0.17     21269
weighted avg       0.59      0.62      0.49     21269

Recall: 0.21008561003989912


### Random Forest

In [116]:
rf = RandomForestClassifier(random_state = 10,
                            n_estimators = 30, 
                               min_samples_split =5)
  #                           
   #                         class_weight = 'balanced', 
    #                        max_depth = 7

In [117]:
rf.fit(X1_train, Y1_train)
print(classification_report(Y1_train, rf.predict(X1_train), np.unique(Y1_train)))

              precision    recall  f1-score   support

     Butters       0.97      0.69      0.81      1821
     Cartman       0.96      0.79      0.87      6842
        Kyle       0.92      0.68      0.78      4969
      Others       0.86      0.99      0.92     30619
        Stan       0.93      0.69      0.79      5376

    accuracy                           0.89     49627
   macro avg       0.93      0.77      0.83     49627
weighted avg       0.89      0.89      0.88     49627



In [118]:
print(classification_report(Y1_test, rf.predict(X1_test),np.unique(Y1_test))) 
print("Recall:",recall_score(Y1_test, rf.predict(X1_test), average = 'macro'))

              precision    recall  f1-score   support

     Butters       0.40      0.10      0.16       781
     Cartman       0.45      0.24      0.31      2932
        Kyle       0.34      0.16      0.22      2130
      Others       0.69      0.90      0.78     13122
        Stan       0.36      0.19      0.25      2304

    accuracy                           0.63     21269
   macro avg       0.45      0.32      0.34     21269
weighted avg       0.57      0.63      0.58     21269

Recall: 0.3189946142692536


In [119]:
rf.fit(X2_train, Y2_train)
print(classification_report(Y2_train, rf.predict(X2_train), np.unique(Y2_train)))

              precision    recall  f1-score   support

     Butters       0.96      0.72      0.82      1821
     Cartman       0.94      0.81      0.87      6842
        Kyle       0.93      0.70      0.80      4969
      Others       0.87      0.99      0.93     30619
        Stan       0.93      0.71      0.81      5376

    accuracy                           0.89     49627
   macro avg       0.93      0.79      0.85     49627
weighted avg       0.90      0.89      0.89     49627



In [120]:
print(classification_report(Y2_test, rf.predict(X2_test),np.unique(Y2_test))) 
print("Recall:",recall_score(Y2_test, rf.predict(X2_test), average = 'macro'))

              precision    recall  f1-score   support

     Butters       0.37      0.07      0.12       781
     Cartman       0.46      0.24      0.31      2932
        Kyle       0.40      0.15      0.22      2130
      Others       0.68      0.92      0.79     13122
        Stan       0.40      0.18      0.25      2304

    accuracy                           0.64     21269
   macro avg       0.46      0.31      0.34     21269
weighted avg       0.58      0.64      0.58     21269

Recall: 0.31332663595758514


### Logistic Regression

In [109]:
lr = LogisticRegression()
#random_state = 10,
 #                       class_weight = 'balanced',
  #                      C = 0.5, 
   #                     max_iter = 3000

In [110]:
lr.fit(X1_train, Y1_train)
print(classification_report(Y1_train, lr.predict(X1_train), np.unique(Y1_train)))

/Users/supriya/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/supriya/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

     Butters       0.89      0.24      0.38      1821
     Cartman       0.76      0.41      0.53      6842
        Kyle       0.74      0.23      0.35      4969
      Others       0.72      0.98      0.83     30619
        Stan       0.67      0.28      0.39      5376

    accuracy                           0.72     49627
   macro avg       0.75      0.43      0.50     49627
weighted avg       0.73      0.72      0.68     49627



In [111]:
print(classification_report(Y1_test, lr.predict(X1_test),np.unique(Y1_test))) 
print("Recall:",recall_score(Y1_test, lr.predict(X1_test), average = 'macro'))

              precision    recall  f1-score   support

     Butters       0.62      0.10      0.18       781
     Cartman       0.55      0.26      0.36      2932
        Kyle       0.49      0.14      0.21      2130
      Others       0.68      0.95      0.80     13122
        Stan       0.44      0.17      0.24      2304

    accuracy                           0.66     21269
   macro avg       0.55      0.32      0.36     21269
weighted avg       0.62      0.66      0.59     21269

Recall: 0.3247072613368493


In [112]:
lr.fit(X2_train, Y2_train)
print(classification_report(Y2_train, lr.predict(X2_train), np.unique(Y2_train)))

              precision    recall  f1-score   support

     Butters       0.86      0.07      0.13      1821
     Cartman       0.60      0.25      0.35      6842
        Kyle       0.67      0.14      0.23      4969
      Others       0.68      0.97      0.80     30619
        Stan       0.58      0.17      0.27      5376

    accuracy                           0.67     49627
   macro avg       0.68      0.32      0.36     49627
weighted avg       0.66      0.67      0.60     49627



In [113]:
print(classification_report(Y2_test, lr.predict(X2_test),np.unique(Y2_test))) 
print("Recall:",recall_score(Y2_test, lr.predict(X2_test), average = 'macro'))

              precision    recall  f1-score   support

     Butters       0.71      0.04      0.08       781
     Cartman       0.52      0.18      0.27      2932
        Kyle       0.57      0.11      0.18      2130
      Others       0.67      0.97      0.79     13122
        Stan       0.48      0.14      0.22      2304

    accuracy                           0.65     21269
   macro avg       0.59      0.29      0.31     21269
weighted avg       0.62      0.65      0.57     21269

Recall: 0.28953763593129833


### Support Vector Machine

In [87]:
sv = LinearSVC(random_state = 10,
                class_weight = 'balanced',
                C = 0.5,
               max_iter = 3000
              )          
#class_weight = {'Butters': 0.5, 'Cartman': 0.9, 'Kyle': 0.5, 'Others': 0.1, 'Stan': 0.5}

In [88]:
sv.fit(X1_train, Y1_train)
print(classification_report(Y1_train, sv.predict(X1_train), np.unique(Y1_train)))

              precision    recall  f1-score   support

     Butters       0.78      0.88      0.83      1821
     Cartman       0.95      0.87      0.91      6842
        Kyle       0.88      0.80      0.84      4969
      Others       0.92      0.96      0.94     30619
        Stan       0.90      0.79      0.84      5376

    accuracy                           0.91     49627
   macro avg       0.89      0.86      0.87     49627
weighted avg       0.91      0.91      0.91     49627



In [89]:
print(classification_report(Y1_test, sv.predict(X1_test),np.unique(Y1_test))) 
print("Recall:",recall_score(Y1_test, sv.predict(X1_test), average = 'micro'))

              precision    recall  f1-score   support

     Butters       0.20      0.20      0.20       781
     Cartman       0.43      0.31      0.36      2932
        Kyle       0.29      0.23      0.26      2130
      Others       0.73      0.82      0.77     13122
        Stan       0.31      0.26      0.28      2304

    accuracy                           0.61     21269
   macro avg       0.39      0.36      0.37     21269
weighted avg       0.58      0.61      0.59     21269

Recall: 0.6060933753349946


In [90]:
sv.fit(X2_train, Y2_train)
print(classification_report(Y2_train, sv.predict(X2_train), np.unique(Y2_train)))

              precision    recall  f1-score   support

     Butters       0.70      0.89      0.78      1821
     Cartman       0.91      0.88      0.90      6842
        Kyle       0.84      0.81      0.82      4969
      Others       0.93      0.94      0.94     30619
        Stan       0.85      0.80      0.83      5376

    accuracy                           0.90     49627
   macro avg       0.85      0.87      0.85     49627
weighted avg       0.90      0.90      0.90     49627



In [91]:
print(classification_report(Y2_test, sv.predict(X2_test),np.unique(Y2_test))) 
print("Recall:",recall_score(Y2_test, sv.predict(X2_test), average = 'micro'))

              precision    recall  f1-score   support

     Butters       0.22      0.19      0.20       781
     Cartman       0.46      0.28      0.34      2932
        Kyle       0.32      0.21      0.25      2130
      Others       0.71      0.86      0.78     13122
        Stan       0.34      0.23      0.27      2304

    accuracy                           0.62     21269
   macro avg       0.41      0.35      0.37     21269
weighted avg       0.58      0.62      0.59     21269

Recall: 0.6213738304574733


### End Of Document